# Fuzzing with Constraints

In previous chapters, we have seen how [Grammar-Based Fuzzing](GrammarFuzzer.ipynb) allows us to express the structure of an input using context-free grammars, and how we can use these very grammars to efficiently generate myriads of valid inputs.
However, there are input features that cannot be expressed in a context-free language: "$X$ is the length of $Y$", for instance, or "$X$ is an identifier previously declared".
In this chapter, we show how the *ISLa* framework allows us to express such features as _constraints_ added to a grammar, and how ISLa solves these constraints automatically to produce inputs that are not only _syntactically_ valid, but actually _semantically_ valid.

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* You should have read the [chapter on grammars](Grammars.ipynb).
* The chapter on [generators and filters](GeneratorGrammarFuzzer.ipynb) addresses a similar problem, but with program code instead of constraints.

In [ ]:
import bookutils

In [ ]:
# ignore
import sys

In [ ]:
# ignore
if sys.version_info < (3, 10):
    print("This code requires Python 3.10 or later")
    sys.exit(0)

## Semantic Input Properties

In this book, we have frequently used [grammars](Grammars.ipynb) to [systematically generate inputs](GrammarFuzzer.ipynb) that [cover input structure](GrammarCoverageFuzzer.ipynb) and more.
But while it is relatively easy to express the _syntax_ of an input using a grammar, there are input properties that _cannot_ be expressed using a grammar.
Such input properties are called _semantic_ properties.

Let us illustrate semantic properties using a simple example.
We want to test some system that is configured by two settings, a _page size_ and a _buffer size_. 
Both these come as integer numbers as part of a human-readable configuration file.
The _syntax_ of this file is given by the following grammar:

In [ ]:
from Grammars import Grammar, is_valid_grammar

In [ ]:
import string

In [ ]:
CONFIG_GRAMMAR: Grammar = {
    "<start>": ["<config>"],
    "<config>": [
        "pagesize=<pagesize>\n"
        "bufsize=<bufsize>"
    ],
    "<pagesize>": ["<int>"],
    "<bufsize>": ["<int>"],
    "<int>": ["<digit>", "<leaddigit><int>"],
    "<digit>": list("0123456789"),
    "<leaddigit>": list("123456789")
}

In [ ]:
assert is_valid_grammar(CONFIG_GRAMMAR)

Using this grammar, we can now use any of our grammar-based fuzzers to generate valid inputs.
For instance:

In [ ]:
from GrammarFuzzer import GrammarFuzzer

In [ ]:
fuzzer = GrammarFuzzer(CONFIG_GRAMMAR)

In [ ]:
for i in range(10):
    print(i)
    print(fuzzer.fuzz())

So far, so good - and indeed, these random values will help us test our (hypothetical) system.
But what if we want to _control_ these values further, putting our system to the test?

A grammar gives us _some_ control. If we want to ensure a page size of at least 100,000, for instance, a rule like
```python
"<bufsize>": ["<leaddigit><digit><digit><digit><digit><digit>"]
```
would do the job.
We could also express that the page size should be an odd number, by having it end in an odd digit.
But if we want to state that the page size should be, say, a multiple of 8, or larger or less than the buffer size, we are out of luck.

In the [chapter on fuzzing with generators](GeneratorGrammarFuzzer.ipynb), we have seen how to attach _program code_ to individual rules - program code that would either _generate_ individual elements right away or _filter_ only these that satisfy specific conditions.

Attaching code makes things very flexible, but also has several disadvantages:

* First, it is pretty hard to generate inputs that satisfy multiple constraints at once.
In essence, you have to code your own _strategy_ for generating inputs, which at some point negates the advantage of having an abstract representation such as a grammar.
* Second, your code is not portable. While a grammar can be easily adapted to _any_ grammar-based fuzzer, adding, say, Python code, ties you to the Python environment forever.
* Third, program code can only be used for _producing_ inputs or _checking_ inputs, but not both. This, again, is a downside compared to a pure grammar representation.

Another, very general solution to this problem would be to use _unrestricted_ grammars rather than the _context-free_ grammars we have used so far.
In an unrestricted grammar, one can have multiple symbols also on the left hand side of an expansion rule, making them very flexible.
In fact, they are _Turing-universal_, meaning that they can express any feature that could also be expressed in program code; and they could thus check and produce arbitrary strings with arbitrary features. (If they finish, that is – unrestricted grammars also suffer from the halting problem.)

The downside is that there is literally no programming support for unrestricted grammars – we'd have to implement all of arithmetics, strings, and other functionality from scratch in a grammar, which is - well - not fun.

## Constraints to the Rescue

In recent work, Dominic Steinhöfel and Andreas Zeller (one of the authors of this book) have presented an infrastructure that allows to produce inputs with _arbitrary properties_, but without having to go through the trouble of implementing producers or checkers.
Instead, they suggest a dedicated _language_ for specifiying inputs, named _ISLa_ (for input specification language).
_ISLa_ combines a standard context-free _grammar_ with _constraints_ that express _semantic_ properties of the inputs and their elements.
ISLa can be used as a _fuzzer_ (producing inputs that satisfy the constraints) as well as a _checker_ (checking inputs whether they satisfy the given constraints).

Let us illustrate ISLa by example. ISLa comes as a Python package named `isla-solver` that can be easily installed using `pip`:

```shell
$ pip install isla-solver
```

This also installs all dependent packages (such as `fuzzingbook`, for instance).

The core of ISLa is the _ISLa Solver_ – the component that actually _solves_ constraints to produce satisfying inputs.

In [ ]:
import isla  # type: ignore

In [ ]:
from isla.language import parse_isla  # type: ignore

In [ ]:
from isla.solver import ISLaSolver  # type: ignore

The constructor of an `ISLASolver` takes two mandatory arguments.
* The _grammar_ is the grammar the solver should produce inputs from.
* The _constraint_ is the constraint the produced inputs should satisfy.

To express a constraint, we have a variety of _functions_ and _predicates_ at our disposition.
These can be applied to individual elements of the grammar, notably their nonterminals.
The function `str.len()`, for instance, returns the length of a string.
If we want to have inputs in which the page size has at least 6 digits, we can write:

In [ ]:
solver = ISLaSolver(CONFIG_GRAMMAR, 'str.len(<pagesize>) >= 6')

The method `solve()` returns an iterator over all produced strings from the ISLa solver. To get these inputs, we can thus write

In [ ]:
for i in range(10):
    print(i)
    print(next(solver.solve()))   

... and we see that, indeed, each page size has exactly six digits.

To express a minimum numeric value, however, we can use a more elegant way.
The function `str.to_int()`, for instance, converts a string into an integer.
To obtain a page size that of at least 100000, we can thus also write

In [ ]:
solver = ISLaSolver(CONFIG_GRAMMAR, 'str.to.int(<pagesize>) >= 100000')
print(next(solver.solve()))

If we want the page size to be in the range of 100 to 200, we can state this as a logical conjunction (using `and`)

In [ ]:
solver = ISLaSolver(CONFIG_GRAMMAR, 
                    '''
                    str.to.int(<pagesize>) >= 100 and 
                    str.to.int(<pagesize>) <= 200
                    ''')
print(next(solver.solve()))

And if we want the page size to be a multiple of seven, we can write

In [ ]:
solver = ISLaSolver(CONFIG_GRAMMAR, 
                    '''
                    (= (mod (str.to.int <pagesize>) 7) 0)
                    ''')
print(next(solver.solve()))

* FIXME: How do I write this in infix syntax?
* FIXME: Suppress warnings unless relevant for the user

ISLa constraints can also involve multiple elements. Expressing equality between two elements is easy:

In [ ]:
solver = ISLaSolver(CONFIG_GRAMMAR, 
                    '''
                    <pagesize> = <bufsize>
                    ''')
print(next(solver.solve()))

We can also use numerical constraints, stating that the buffer size should always be exactly one more than the page size:

In [ ]:
solver = ISLaSolver(CONFIG_GRAMMAR, 
                    '''
                    str.to.int(<pagesize>) > 1024 and
                    str.to.int(<bufsize>) = str.to.int(<pagesize>) + 1
                    ''')
print(next(solver.solve()))

All the above functions (like `str.to.int`) actually stem from the _SMT-LIB_ library for _satisfiability modulo theories_ (SMT), a standard for expressing constraints for constraint solvers (like ISLa).
The [list of all theories defined in SMT-LIB](https://smtlib.cs.uiowa.edu/theories.shtml) lists dozens of functions and predicates that can be used in ISLa constraints.

### Excursion: Using SMT-LIB Syntax

Instead of the above "infix" syntax which is familiar to programmers, ISLa also supports full SMT-LIB syntax.
Instead of writing $f(x_1, x_2, x_3, \dots)$ for a function $f$ and its arguments $x_1 \dots x_n$, SMT-LIB uses a "prefix" LISP-like syntax in which all functions and operators are written as $(f x_1 x_2 x_3 \dots)$.
The above predicate would thus be written as

In [ ]:
solver = ISLaSolver(CONFIG_GRAMMAR, 
                '''
                (> (str.to.int <pagesize>) 1024)
                and
                (= (str.to.int <bufsize>) (+ (str.to.int <pagesize>) 1))
                ''')
print(next(solver.solve()))

Note that for boolean operators such as `and`, we still use the ISLa infix syntax; having ISLa handle these operators is more efficient than passing them on to the constraint solver.

* FIXME: Support `and` as part of SMT-LIB expressions

### End of Excursion

## Accessing Elements

Have a look at XPath syntax

## Quantifiers

The `exists` quantifier

## Checking Inputs

The `evaluate()` method

## A Simple Language

In [ ]:
LANG_GRAMMAR: Grammar = {
    "<start>":
        ["<stmt>"],
    "<stmt>":
        ["<assgn>", "<assgn> ; <stmt>"],
    "<assgn>":
        ["<var> := <rhs>"],
    "<rhs>":
        ["<var>", "<digit>"],
    "<var>": list(string.ascii_lowercase),
    "<digit>": list(string.digits)
}

In [ ]:
assert is_valid_grammar(LANG_GRAMMAR)

## A Simple Fuzzer

In [ ]:
for i in range(10):
    print(fuzzer.fuzz())

## Beyond Syntax: Semantic Input Properties

Let's start with ISLa.

In [ ]:
from isla.isla_predicates import STANDARD_STRUCTURAL_PREDICATES  # type: ignore

Let's try the new syntax.

In [ ]:
constraint = parse_isla("""
exists <assgn> assgn: 
  (before(assgn, <assgn>) and <assgn>.<rhs>.<var> = assgn.<var>)
""",
grammar=LANG_GRAMMAR,
structural_predicates=STANDARD_STRUCTURAL_PREDICATES)

Okay. Something simpler, maybe?

In [ ]:
constraint = parse_isla("""
exists <digit> x:
  (str.to.int(x) >= 9)
""", grammar=LANG_GRAMMAR)

How about the old SMT-LIB syntax?

In [ ]:
constraint = parse_isla("""
exists <digit> x:
  (>= (str.to.int x) 9)
""", LANG_GRAMMAR)

Ok, this one works :-)

In [ ]:
constraint = parse_isla("""
  (>= (str.to.int <digit>) 9)
""", LANG_GRAMMAR)

In [ ]:
solver = ISLaSolver(
    grammar=LANG_GRAMMAR,
    formula=constraint)

In [ ]:
solver = ISLaSolver(LANG_GRAMMAR, '(>= (str.to.int <digit>) 9)')

In [ ]:
for i in range(10):
    print(next(solver.solve()))

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

In [ ]:
# ignore
import warnings

In [ ]:
# ignore
with warnings.catch_warnings():
    warnings.simplefilter("default")

    from ClassDiagram import display_class_hierarchy
    hierarchy = display_class_hierarchy([ISLaSolver],
                           public_methods=[
                                ISLaSolver.__init__,
                                ISLaSolver.solve,
                            ])
hierarchy

* FIXME: Have docstrings for publicly available methods
* FIXME: Have a docstring for the `ISLaSolver` class
* FIXME: How do I check a given string whether it satisfies constraints? (Likely `isla.evaluator.evaluate(constraint, tree, grammar)`)

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_